In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import io
import numpy as np
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # needed for waffle Charts

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

#!pip install folium
import folium # map rendering library

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
print('Libraries imported')

Matplotlib version:  3.3.3
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



geopy-

In [2]:
# set some defaults 
default_colormap = 'RdYlBu_r'

In [3]:
#parse the Excel data
amsterdam_data = pd.read_excel('GEBIED_BUURTEN.xlsx')

# remove first header row
amsterdam_data.columns = amsterdam_data.iloc[0]
amsterdam_data = amsterdam_data.reindex(amsterdam_data.index.drop(0)).reset_index(drop=True)
amsterdam_data.columns.name = None

# drop obsolete columns
amsterdam_data.drop(columns=['OBJECTNUMMER','Buurt_code','Buurtcombinatie_code','Opp_m2','WKT_LNG_LAT', 'WKT_LAT_LNG'],inplace=True)

# rename from Dutch to international terms
amsterdam_data.rename(columns={"Buurt": "Neighbourhood"},inplace=True)
amsterdam_data.rename(columns={"Stadsdeel_code": "Borough"},inplace=True)


# rename for better readability
amsterdam_data.rename(columns={"LAT": "Latitude"},inplace=True)
amsterdam_data.rename(columns={"LNG": "Longitude"},inplace=True)

# focus on center of Amsterdam, filter out other boroughs
amsterdam_data = amsterdam_data[np.isin(amsterdam_data['Borough'], ['A'])]

amsterdam_data
#amsterdam_data.shape


,Neighbourhood,Borough,Longitude,Latitude,NaN
36,Kop Zeedijk,A,4.9001715,52.3757235,NaN
37,BG-terrein e.o.,A,4.89557815,52.369559,NaN
38,Stationsplein e.o.,A,4.9009435,52.3797652,NaN
39,Hemelrijk,A,4.8949027,52.37821835,NaN
40,Spuistraat Noord,A,4.8915324,52.37508835,NaN
...,...,...,...,...,...
354,Marine-Etablissement,A,4.9160864,52.37427965,NaN
355,Oostenburg,A,4.9260914,52.3706328,NaN
356,Czaar Peterbuurt,A,4.928627,52.3696181,NaN
357,Het Funen,A,4.92984555,52.36849275,NaN


In [4]:
address = 'Amsterdam, Netherlands'

geolocator = Nominatim(user_agent="ams_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Amsterdam are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Amsterdam are 52.3727598, 4.8936041.


In [5]:
# create map of New York using latitude and longitude values
#map_amsterdam = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
#for lat, lng, borough, neighbourhood in zip(amsterdam_data['Latitude'], amsterdam_data['Longitude'], amsterdam_data['Borough'], amsterdam_data['Neighbourhood']):
#    label = '{}, {}'.format(neighbourhood, borough)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(map_amsterdam)  
    
#map_amsterdam

In [6]:
# Foursquare credentials # @hidden_cell
CLIENT_ID = 'BOXVYZAJBVPSF10JV0A43WVLTD2BBD3GFMWGMFXUQECKTZ42' # your Foursquare ID
CLIENT_SECRET = 'UTCLKHZDFGNIYALQOSVE1QR3HJTF3IN51EHPK3AGEUXP4N40' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

radius = 2000
section = 'food'

exclude_categories = ['Park','Fitness','Art Museum','Gym','Fitness Center']

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [7]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
# helper function to parse the results from an exploration of a list of locations
def getNearbyVenues(names, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
       
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            section,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        print(name,'has',len(results),'number of venues')
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    

    
    return(nearby_venues)

In [9]:
amsterdam_venues = getNearbyVenues(names=amsterdam_data['Neighbourhood'],
                                   latitudes=amsterdam_data['Latitude'],
                                   longitudes=amsterdam_data['Longitude']
                                  )


print('number of venues before de-duplication', amsterdam_venues.shape)
amsterdam_venues.drop_duplicates('Venue',inplace=True,ignore_index=true)
print('number of venues after de-duplication', amsterdam_venues.shape)
amsterdam_venues


Kop Zeedijk has 100 number of venues
BG-terrein e.o. has 100 number of venues
Stationsplein e.o. has 100 number of venues
Hemelrijk has 100 number of venues
Spuistraat Noord has 100 number of venues
Nieuwe Kerk e.o. has 100 number of venues
Spuistraat Zuid has 100 number of venues
Kalverdriehoek has 100 number of venues
Langestraat e.o. has 100 number of venues
Leliegracht e.o. has 100 number of venues
Felix Meritisbuurt has 100 number of venues
Leidsegracht Noord has 100 number of venues
Spiegelbuurt has 100 number of venues
Gouden Bocht has 100 number of venues
Van Loonbuurt has 100 number of venues
Amstelveldbuurt has 100 number of venues
Rembrandtpleinbuurt has 100 number of venues
Scheepvaarthuisbuurt has 100 number of venues
Rapenburg has 100 number of venues
Lastage has 100 number of venues
Uilenburg has 100 number of venues
Valkenburg has 100 number of venues
Zuiderkerkbuurt has 100 number of venues
Waterloopleinbuurt has 100 number of venues
Haarlemmerbuurt West has 100 number

NameError: name 'true' is not defined

In [ ]:
amsterdam_venues['Venue Category'].value_counts().to_frame()

In [ ]:
# check the list of venues per neighbourhood
amsterdam_venues.groupby('Neighbourhood').count()

In [ ]:
# find out how many unique categories we have
print('There are {} uniques categories.'.format(len(amsterdam_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding to do analysis on categorical variables
amsterdam_onehot = pd.get_dummies(amsterdam_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
amsterdam_onehot['Neighbourhood'] = amsterdam_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [amsterdam_onehot.columns[-1]] + list(amsterdam_onehot.columns[:-1])
amsterdam_onehot = amsterdam_onehot[fixed_columns]

amsterdam_onehot.head()

In [ ]:
# group by neighbourhood
amsterdam_grouped = amsterdam_onehot.groupby('Neighbourhood').mean().reset_index()
amsterdam_grouped

In [ ]:
# print top 5 list of venues for every neighbourhood
num_top_venues = 5

for hood in amsterdam_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = amsterdam_grouped[amsterdam_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
# install wordcloud
!pip install wordcloud

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud is installed and imported!')

In [ ]:
# instantiate a word cloud object to visualize all the categories
categories_wc = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=['Shop','Restaurant'],
    width=1600,
    height=800
)


words = amsterdam_venues['Venue Category'].str.cat(sep=' ')
#words

# generate the word cloud
categories_wc.generate(words)

# display the word cloud
plt.figure( figsize=(40,10) )
plt.imshow(categories_wc, interpolation='bilinear')
plt.title('Restaurant categories in Amsterdam',fontdict={'fontsize':14})
plt.axis('off')
plt.savefig('ams_wordcloud.png')
plt.show()

In [ ]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
newyork_neighbourhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
newyork_neighbourhoods = pd.DataFrame(columns=column_names)

for data in newyork_neighbourhoods_data:
    borough = neighbourhood_name = data['properties']['borough'] 
    neighbourhood_name = data['properties']['name']
        
    neighbourhood_latlon = data['geometry']['coordinates']
    neighbourhood_lat = neighbourhood_latlon[1]
    neighbourhood_lon = neighbourhood_latlon[0]
    
    newyork_neighbourhoods = newyork_neighbourhoods.append({'Borough': borough,
                                          'Neighbourhood': neighbourhood_name,
                                          'Latitude': neighbourhood_lat,
                                          'Longitude': neighbourhood_lon}, ignore_index=True)
    
newyork_neighbourhoods  





In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(newyork_neighbourhoods['Borough'].unique()),
        newyork_neighbourhoods.shape[0]
    )
)

In [ ]:
# only get the data from Manhattan
manhattan_data = newyork_neighbourhoods[newyork_neighbourhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data

In [ ]:
# get list of venues
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighbourhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

print('number of venues in Manhattan before de-duplication', manhattan_venues.shape)
manhattan_venues.drop_duplicates('Venue',inplace=True,ignore_index)
print('number of venues in Manhattan after de-duplication', manhattan_venues.shape)
manhattan_venues


In [ ]:
# instantiate a word cloud object to visualize all the categories in Manhattan
newyork_categories_wc = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=['Shop','Restaurant'],
    width=1600,
    height=800
)


words = manhattan_venues['Venue Category'].str.cat(sep=' ')

# generate the word cloud
newyork_categories_wc.generate(words)

# display the word cloud
plt.figure( figsize=(40,10) )
plt.imshow(newyork_categories_wc, interpolation='bilinear')
plt.title('Restaurant categories in New York',fontdict={'fontsize':14})
plt.axis('off')
plt.savefig('new_york_wordcloud.png')
plt.show()

In [ ]:
ams_category_counts = amsterdam_venues['Venue Category'].value_counts().to_frame()


# Make the colum name better readable
ams_category_counts.rename(columns = {'Venue Category':'Count'}, inplace = True)

ams_category_counts.reset_index(inplace=True)
ams_category_counts.rename(columns = {'index':'Category'}, inplace = True)

print('Number of different sampled restaurants in Amsterdam is' , ams_category_counts['Count'].count())
print('Total number of sampled restaurants in Amsterdam is' , ams_category_counts['Count'].sum())

ams_category_counts

In [ ]:
ny_category_counts = manhattan_venues['Venue Category'].value_counts().to_frame()

ny_category_counts.rename(columns = {'Venue Category':'Count'}, inplace = True)
ny_category_counts.reset_index(inplace=True)
ny_category_counts.rename(columns = {'index':'Category'}, inplace = True)

print('Number of different sampled restaurant categories in New York is' , ny_category_counts['Count'].count())
print('Total number of sampled restaurants in New York is' , ny_category_counts['Count'].sum())

ny_category_counts

In [ ]:
ny_ams_merged_counts = pd.merge(ams_category_counts, ny_category_counts, on="Category",how="outer",)

ny_ams_merged_counts.rename(columns = {'Count_x':'Amsterdam','Count_y':'New York'}, inplace = True)
ny_ams_merged_counts = ny_ams_merged_counts.fillna(0)

# count which categories only appear in each city
ny_ams_merged_counts['ams_only'] = ny_ams_merged_counts.apply(lambda x : 1 if (x['Amsterdam'] >= 0 and x['New York'] == 0) else 0, axis=1)
ny_ams_merged_counts['ny_only'] = ny_ams_merged_counts.apply(lambda x : 1 if (x['New York'] >= 0 and x['Amsterdam'] == 0) else 0, axis=1)

ny_ams_merged_counts.set_index("Category", inplace = True)

ny_ams_merged_counts

In [ ]:
ams_only_count = len(ny_ams_merged_counts[(ny_ams_merged_counts['ams_only'] == 1)])
ny_only_count = len(ny_ams_merged_counts[(ny_ams_merged_counts['ny_only'] == 1)])

print('New York has', ny_only_count, 'categories that are not in Amsterdam')
print('Amsterdam has', ams_only_count, 'categories that are not in New York')

In [ ]:
# to be able to compare numbers, make sure both columns are normalized, aka have values in the same range. As we interested in the relative values, use a percentage. Multiple with 100 to have an easily readable number

ny_ams_merged_counts["Amsterdam"]=((ny_ams_merged_counts["Amsterdam"]-ny_ams_merged_counts["Amsterdam"].min())/(ny_ams_merged_counts["Amsterdam"].max()-ny_ams_merged_counts["Amsterdam"].min()))*100
ny_ams_merged_counts["New York"]=((ny_ams_merged_counts["New York"]-ny_ams_merged_counts["New York"].min())/(ny_ams_merged_counts["New York"].max()-ny_ams_merged_counts["New York"].min()))*100


ny_ams_merged_counts["Amsterdam"]=(ny_ams_merged_counts["Amsterdam"]/ny_ams_merged_counts["Amsterdam"].sum())*100
ny_ams_merged_counts["New York"]=(ny_ams_merged_counts["New York"]/ny_ams_merged_counts["New York"].sum())*100


print('New York has', ny_ams_merged_counts["New York"].sum(), 'total restaurants after normalization')
print('Amsterdam', ny_ams_merged_counts["Amsterdam"].sum(), 'total restaurants after normalization')


ny_ams_merged_counts.to_csv('newyork_amsterdam_categories.csv')

ny_ams_merged_counts

In [ ]:
ax = ny_ams_merged_counts[['Amsterdam','New York']].plot(kind='barh', title ="Count", figsize=(40, 80), fontsize=14,colormap=default_colormap)


# Despine
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# Switch off ticks
ax.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

# Draw vertical axis lines
vals = ax.get_xticks()
for tick in vals:
  ax.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)


#color=['#D93D66','#2D83A6']
ax.set_xlabel("Category", fontsize=14, weight='bold')
ax.set_ylabel("Count", fontsize=14, weight='bold')
plt.title('Restaurant category counts in Amsterdam and New York',fontdict={'fontsize':14})
plt.box(False)
plt.savefig('nyc_ams_bar.png')
plt.show()

In [ ]:
# To make the chart better readable try to summarize some of the categories into regions

# build a dataframe linking categories with regions

regions_data = [['Restaurant','General'],
                ['Café','General'],
                ['Italian Restaurant','European'],
                ['French Restaurant','European'],
                ['Sandwich Place','General'],
                ['Bakery','General'],
                ['Pizza Place','General'],
                ['Deli / Bodega','General'],
                ['Breakfast Spot','General'],
                ['Steakhouse','General'],
                ['Vegetarian / Vegan Restaurant','General'],
                ['Turkish Restaurant','Middle East'],
                ['Thai Restaurant','SouthEast Asia'],
                ['Burger Joint','General'],
                ['Dutch Restaurant','European'],
                ['Seafood Restaurant','General'],
                ['Mediterranean Restaurant','European'],
                ['Asian Restaurant','SouthEast Asia'],
                ['Modern European Restaurant','European'],
                ['Indonesian Restaurant','SouthEast Asia'],
                ['Gastropub','General'],
                ['Bistro','General'],
                ['Greek Restaurant','European'],
                ['Japanese Restaurant','Japanese'],
                ['Tapas Restaurant','European'],
                ['Middle Eastern Restaurant','Middle East'],
                ['Chinese Restaurant','Chinese'],
                ['Sushi Restaurant','Japanese'],
                ['Caribbean Restaurant','Caribbean'],
                ['South American Restaurant','South American'],
                ['Snack Place','General'],
                ['Bagel Shop','General'],
                ['Salad Place','General'],
                ['Vietnamese Restaurant','SouthEast Asia'],
                ['Creperie','European'],
                ['Indian Restaurant','SouthEast Asia'],
                ['Soup Place','General'],
                ['Australian Restaurant','General'],
                ['Friterie','General'],
                ['Food Truck','General'],
                ['Ramen Restaurant','General'],
                ['Tibetan Restaurant','SouthEast Asia'],
                ['Diner','General'],
                ['Ethiopian Restaurant','African'],
                ['Food Court','General'],
                ['Latin American Restaurant','South American'],
                ['Falafel Restaurant','Middle East'],
                ['Buffet','General'],
                ['Portuguese Restaurant','European'],
                ['Scandinavian Restaurant','European'],
                ['Comfort Food Restaurant','General'],
                ['Taco Place','Central American'],
                ['Molecular Gastronomy Restaurant','General'],
                ['Lebanese Restaurant','Middle East'],
                ['Irish Pub','European'],
                ['Doner Restaurant','Middle East'],
                ['Spanish Restaurant','European'],
                ['Fast Food Restaurant','General'],
                ['Korean Restaurant','Korean'],
                ['BBQ Joint','General'],
                ['North Indian Restaurant','SouthEast Asia'],
                ['Fried Chicken Joint','General'],
                ['Udon Restaurant','Japanese'],
                ['Peruvian Restaurant','South American'],
                ['German Restaurant','European'],
                ['African Restaurant','General'],
                ['Mexican Restaurant','Central American'],
                ['Moroccan Restaurant','African'],
                ['Food Stand','General'],
                ['Afghan Restaurant','SouthEast Asia'],
                ['Swiss Restaurant','General'],
                ['Malay Restaurant','SouthEast Asia'],
                ['American Restaurant','American'],
                ['New American Restaurant','American'],
                ['Southern / Soul Food Restaurant','American'],
                ['Cuban Restaurant','Caribbean'],
                ['Donut Shop','General'],
                ['Brazilian Restaurant','South American'],
                ['Szechuan Restaurant','Chinese'],
                ['Hawaiian Restaurant','General'],
                ['Dumpling Restaurant','General'],
                ['Hot Dog Joint','American'],
                ['Empanada Restaurant','Central American'],
                ['Israeli Restaurant','Middle East'],
                ['Taiwanese Restaurant','Chinese'],
                ['Japanese Curry Restaurant','General'],
                ['Arepa Restaurant','South American'],
                ['Noodle House','Japanese'],
                ['Argentinian Restaurant','South American'],
                ['Eastern European Restaurant','European'],
                ['Burrito Place','Central American'],
                ['Poke Place','American'],
                ['Pet Café','General'],
                ['Austrian Restaurant','European'],
                ['Kosher Restaurant','General'],
                ['Wings Joint','American'],
                ['Food','General'],
                ['Czech Restaurant','European'],
                ['Russian Restaurant','General'],
                ['Dim Sum Restaurant','Chinese'],
                ['Soba Restaurant','Japanese'],
                ['Cantonese Restaurant','Chinese'],
                ['Filipino Restaurant','SouthEast Asia'],
                ['Cafeteria','General'],
                ['Halal Restaurant','Middle East'],
                ['Theme Restaurant','General'],
                ['English Restaurant','European'],
                ['Kebab Restaurant','Middle East'],
                ['Churrascaria','South American'],
                ['Puerto Rican Restaurant','Caribbean'],
                ['Shanghai Restaurant','Chinese'],
                ['Hotpot Restaurant','Chinese'],
                ['Ukrainian Restaurant','European'],
                ['Jewish Restaurant','Middle East'],
                ['Persian Restaurant','Middle East'],
                ['Venezuelan Restaurant','South American'],
                ['Gluten-free Restaurant','General']]
        
  
# Create the pandas DataFrame 
regions = pd.DataFrame(regions_data, columns = ['Category', 'Region'])
regions

In [ ]:
ny_ams_merged_regions = pd.merge(ny_ams_merged_counts, regions, on="Category",how="inner")
ny_ams_merged_regions

In [ ]:
ny_ams_merged_regions_grouped = ny_ams_merged_regions.groupby('Region').sum()

#ny_ams_merged_regions_grouped.set_index("Region", inplace = True)
# we don't need the to see the helper column contain the New York or Amsterdam only counts
ny_ams_merged_regions_grouped.drop(columns=['ams_only','ny_only'],inplace=True)
ny_ams_merged_regions_grouped

In [ ]:
# show the restaurants categorized per region in a bar chart
ax = ny_ams_merged_regions_grouped[['Amsterdam','New York']].plot(kind='barh', title ="Count", figsize=(10,8), fontsize=14,colormap=default_colormap)
ax.set_xlabel("Region", fontsize=14, weight='bold')
ax.set_ylabel("Count", fontsize=14, weight='bold')

# Despine
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# Switch off ticks
ax.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

# Draw vertical axis lines
vals = ax.get_xticks()
for tick in vals:
  ax.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)



plt.title('Restaurants by region in Amsterdam and New York',fontdict={'fontsize':14})
plt.gcf().subplots_adjust(left=0.15)
plt.tight_layout()
plt.box(False)
plt.savefig('nyc_ams_regions_bar.png')
plt.show()